# NNAST Model Training on Google Colab

This notebook trains the NNAST model using GPU acceleration.


## 1. 環境セットアップ


In [ ]:
# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


In [ ]:
# リポジトリのクローンまたはコードのアップロード
import os
import zipfile

# 方法1: Gitリポジトリからクローン（推奨）
# !git clone https://github.com/yourusername/nnast_model.git
# os.chdir('nnast_model')

# 方法2: zipファイルをアップロードして解凍
# from google.colab import files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"Extracted {filename}")


In [ ]:
# 依存関係のインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch-geometric
!pip install transformers
!pip install tqdm


## 2. データセットのアップロード


In [ ]:
# 方法1: Google Driveからマウント（推奨）
from google.colab import drive
drive.mount('/content/drive')

# データセットをGoogle Driveにアップロードしてから、ここでパスを指定
# 例: /content/drive/MyDrive/nnast_dataset/training_data
DATASET_BASE = "/content/drive/MyDrive/nnast_dataset"


In [ ]:
# 方法2: 直接アップロード（小規模データセット用）
# from google.colab import files
# uploaded = files.upload()
# 
# import zipfile
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"Extracted {filename}")


## 3. データセットパスの確認と自動検出


In [ ]:
# Google Drive内のデータセットディレクトリを自動検出
import glob

def find_dataset_directory():
    """Google Drive内でデータセットディレクトリを検索"""
    drive_base = "/content/drive/MyDrive"
    
    if not os.path.exists(drive_base):
        print("⚠️ Google Driveがマウントされていません。Cell 6を先に実行してください。")
        return None, None
    
    # 検索パターン（優先順位順）
    patterns = [
        "**/nnast_colab_dataset/training_data/train_graphs.jsonl",
        "**/nnast_dataset/training_data/train_graphs.jsonl",
        "**/training_data/train_graphs.jsonl",
        "**/train_graphs.jsonl",
    ]
    
    print("データセットファイルを検索中...")
    for pattern in patterns:
        full_pattern = os.path.join(drive_base, pattern)
        matches = glob.glob(full_pattern, recursive=True)
        if matches:
            graphs_file = matches[0]
            print(f"✓ 見つかりました: {graphs_file}")
            # training_dataディレクトリを取得
            dataset_dir = os.path.dirname(graphs_file)
            base_dir = os.path.dirname(dataset_dir)
            return base_dir, graphs_file
    
    return None, None

# DATASET_BASEが定義されていない場合のデフォルト値
if 'DATASET_BASE' not in globals():
    DATASET_BASE = "/content/drive/MyDrive/nnast_dataset"

# データセットディレクトリを検索
found_base, found_graphs = find_dataset_directory()

if found_base:
    print(f"\n✓ データセットが見つかりました: {found_base}")
    DATASET_BASE = found_base
    print(f"  ベースディレクトリを更新: {DATASET_BASE}")
else:
    print(f"\n⚠️ 自動検出でデータセットが見つかりませんでした。")
    print(f"  現在の設定: {DATASET_BASE}")
    print(f"\nGoogle Drive内のディレクトリ一覧:")
    
    # Google Drive内のディレクトリ一覧を表示
    drive_base = "/content/drive/MyDrive"
    if os.path.exists(drive_base):
        items = os.listdir(drive_base)
        if items:
            for item in items[:20]:  # 最初の20個のみ表示
                item_path = os.path.join(drive_base, item)
                if os.path.isdir(item_path):
                    print(f"  📁 {item}/")
                    # training_dataディレクトリがあるか確認
                    training_data_path = os.path.join(item_path, "training_data")
                    if os.path.exists(training_data_path):
                        print(f"     └─ training_data/ (見つかりました！)")
                        # ファイルを確認
                        graphs_path = os.path.join(training_data_path, "train_graphs.jsonl")
                        labels_path = os.path.join(training_data_path, "train_labels.jsonl")
                        if os.path.exists(graphs_path):
                            print(f"        ✓ train_graphs.jsonl")
                        if os.path.exists(labels_path):
                            print(f"        ✓ train_labels.jsonl")
        else:
            print("  (ディレクトリが空です)")
    else:
        print("  Google Driveがマウントされていません。")

print()

# データセットファイルのパス設定
GRAPHS_FILE = os.path.join(DATASET_BASE, "training_data", "train_graphs.jsonl")
LABELS_FILE = os.path.join(DATASET_BASE, "training_data", "train_labels.jsonl")

# 見つかったパスを使用
if found_graphs:
    GRAPHS_FILE = found_graphs
    LABELS_FILE = os.path.join(os.path.dirname(found_graphs), "train_labels.jsonl")

print("=" * 60)
print("設定されたパス")
print("=" * 60)
print(f"データセットベース: {DATASET_BASE}")
print(f"グラフファイル: {GRAPHS_FILE}")
print(f"ラベルファイル: {LABELS_FILE}")
print("=" * 60)
print()

# ファイル存在確認
files_ok = True

if os.path.exists(GRAPHS_FILE):
    file_size = os.path.getsize(GRAPHS_FILE) / (1024 * 1024)  # MB
    print(f"✓ train_graphs.jsonl が見つかりました ({file_size:.2f} MB)")
else:
    print(f"✗ train_graphs.jsonl が見つかりません")
    print(f"  パス: {GRAPHS_FILE}")
    files_ok = False

if os.path.exists(LABELS_FILE):
    file_size = os.path.getsize(LABELS_FILE) / (1024 * 1024)  # MB
    print(f"✓ train_labels.jsonl が見つかりました ({file_size:.2f} MB)")
else:
    print(f"✗ train_labels.jsonl が見つかりません")
    print(f"  パス: {LABELS_FILE}")
    files_ok = False

print()

# 両方のファイルが存在する場合のみ学習を続行
if files_ok:
    print("✓ すべてのファイルが準備できました。Cell 11で学習を開始できます。")
else:
    print("⚠️ ファイルが見つかりません。以下のいずれかを実行してください:")
    print("\n【方法1】手動でパスを設定:")
    print("  以下のセルを実行して、実際のパスに置き換えてください:")
    print('  GRAPHS_FILE = "/content/drive/MyDrive/実際のディレクトリ/training_data/train_graphs.jsonl"')
    print('  LABELS_FILE = "/content/drive/MyDrive/実際のディレクトリ/training_data/train_labels.jsonl"')
    print("\n【方法2】ファイルを検索:")
    print("  次のセルでファイルを検索できます:")
    print('  !find /content/drive/MyDrive -name "train_graphs.jsonl" 2>/dev/null')


In [ ]:
# デバッグ: Google Drive内でファイルを検索（必要に応じて実行）
# このセルは、Cell 9でファイルが見つからない場合に実行してください

print("Google Drive内でtrain_graphs.jsonlを検索中...")
!find /content/drive/MyDrive -name "train_graphs.jsonl" -type f 2>/dev/null | head -10

print("\nGoogle Drive内でtrain_labels.jsonlを検索中...")
!find /content/drive/MyDrive -name "train_labels.jsonl" -type f 2>/dev/null | head -10

print("\nもしファイルが見つかった場合、そのパスをコピーして、以下のように設定してください:")
print("GRAPHS_FILE = \"見つかったパス/train_graphs.jsonl\"")
print("LABELS_FILE = \"見つかったパス/train_labels.jsonl\"")


In [ ]:
# 手動でパスを設定する場合（Cell 9で自動検出できなかった場合）
# 以下のパスを実際のGoogle Drive内のパスに置き換えてください

# 例: GRAPHS_FILE = "/content/drive/MyDrive/nnast_colab_dataset/training_data/train_graphs.jsonl"
# 例: LABELS_FILE = "/content/drive/MyDrive/nnast_colab_dataset/training_data/train_labels.jsonl"

# コメントアウトを外して、実際のパスに置き換えてください:
# GRAPHS_FILE = "/content/drive/MyDrive/実際のディレクトリ/training_data/train_graphs.jsonl"
# LABELS_FILE = "/content/drive/MyDrive/実際のディレクトリ/training_data/train_labels.jsonl"

# 設定後、ファイルの存在を確認
# if 'GRAPHS_FILE' in locals() and os.path.exists(GRAPHS_FILE):
#     print(f"✓ GRAPHS_FILE が設定されました: {GRAPHS_FILE}")
# if 'LABELS_FILE' in locals() and os.path.exists(LABELS_FILE):
#     print(f"✓ LABELS_FILE が設定されました: {LABELS_FILE}")


## 4. 学習の実行


In [ ]:
# 学習パラメータ
OUTPUT_DIR = "./checkpoints"

# GPU用の最適化設定
BATCH_SIZE = 16  # GPUなら大きく設定可能（メモリに応じて調整）
EPOCHS = 10
MAX_NODES = 1000
HIDDEN_DIM = 256
LEARNING_RATE = 1e-4

# パスの再確認（Cell 9を実行していない場合に備えて）
try:
    # GRAPHS_FILEとLABELS_FILEが定義されているか確認
    if 'GRAPHS_FILE' not in globals() or 'LABELS_FILE' not in globals():
        print("⚠️ パスが設定されていません。Cell 9を先に実行してください。")
        print("手動でパスを設定する場合:")
        print('  GRAPHS_FILE = "/content/drive/MyDrive/あなたのディレクトリ/training_data/train_graphs.jsonl"')
        print('  LABELS_FILE = "/content/drive/MyDrive/あなたのディレクトリ/training_data/train_labels.jsonl"')
        raise NameError("GRAPHS_FILE or LABELS_FILE not defined")
    
    # ファイル存在確認
    print("=" * 60)
    print("データセットファイルの確認")
    print("=" * 60)
    print(f"グラフファイル: {GRAPHS_FILE}")
    print(f"  存在: {os.path.exists(GRAPHS_FILE)}")
    if os.path.exists(GRAPHS_FILE):
        file_size = os.path.getsize(GRAPHS_FILE) / (1024 * 1024)
        print(f"  サイズ: {file_size:.2f} MB")
    
    print(f"\nラベルファイル: {LABELS_FILE}")
    print(f"  存在: {os.path.exists(LABELS_FILE)}")
    if os.path.exists(LABELS_FILE):
        file_size = os.path.getsize(LABELS_FILE) / (1024 * 1024)
        print(f"  サイズ: {file_size:.2f} MB")
    print("=" * 60)
    print()
    
    # ファイルが存在する場合のみ学習を実行
    if os.path.exists(GRAPHS_FILE) and os.path.exists(LABELS_FILE):
        print("✓ すべてのファイルが準備できました。学習を開始します...")
        print()
        
        !python3 -m ml.train \
            --graphs "{GRAPHS_FILE}" \
            --labels "{LABELS_FILE}" \
            --output-dir "{OUTPUT_DIR}" \
            --batch-size {BATCH_SIZE} \
            --epochs {EPOCHS} \
            --hidden-dim {HIDDEN_DIM} \
            --max-nodes {MAX_NODES} \
            --device cuda \
            --gnn-type GAT \
            --lr {LEARNING_RATE}
    else:
        print("⚠️ データセットファイルが見つかりません。")
        print("\n対処方法:")
        print("1. Cell 9を実行してパスを自動検出してください")
        print("2. または、以下のように手動でパスを設定してください:")
        print()
        print("   # Google Drive内の実際のパスに置き換えてください")
        print('   GRAPHS_FILE = "/content/drive/MyDrive/実際のパス/training_data/train_graphs.jsonl"')
        print('   LABELS_FILE = "/content/drive/MyDrive/実際のパス/training_data/train_labels.jsonl"')
        print()
        print("3. Google Drive内のファイル構造を確認:")
        print("   !ls -la /content/drive/MyDrive/")
        print("   !find /content/drive/MyDrive -name 'train_graphs.jsonl' 2>/dev/null")
        
except NameError as e:
    print(f"エラー: {e}")
    print("\nCell 9を先に実行してください。")


## 5. 結果の確認とダウンロード


In [ ]:
# 学習履歴の可視化
import json
import matplotlib.pyplot as plt

history_file = f"{OUTPUT_DIR}/training_history.json"
if os.path.exists(history_file):
    with open(history_file) as f:
        history = json.load(f)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    
    plt.tight_layout()
    plt.show()
else:
    print("Training history file not found")


In [ ]:
# チェックポイントをGoogle Driveに保存
import shutil

drive_checkpoint_dir = f"{DATASET_BASE}/checkpoints"
os.makedirs(drive_checkpoint_dir, exist_ok=True)

if os.path.exists(OUTPUT_DIR):
    # 既存のディレクトリをコピー
    if os.path.exists(drive_checkpoint_dir):
        shutil.rmtree(drive_checkpoint_dir)
    shutil.copytree(OUTPUT_DIR, drive_checkpoint_dir)
    print(f"✓ Checkpoints saved to {drive_checkpoint_dir}")
else:
    print("Checkpoint directory not found")
